# Preliminares

## Preparación del entorno (solo para ejecutar en Colab) 

Las dos siguiente casillas de código instalan y configuran Valgrind  en el Kernel

In [ ]:
!apt-get install valgrind

In [ ]:
import os, json, signal, time

if 'libtcmalloc' in os.environ.get('LD_PRELOAD', ''):
  kernel_fn = '/usr/local/share/jupyter/kernels/python3/kernel.json'
  spec = json.load(open(kernel_fn))
  spec['env'] = {'LD_PRELOAD': ''}
  json.dump(spec, open(kernel_fn, 'w'))
  print("Installed workaround for valgrind. Don't wait for this cell to finish,")
  print('just REFRESH the browser tab and RUN this cell again.', flush=True)
  time.sleep(0.5)
  # killing the kernel manager so that specs get reloaded
  os.kill(os.getppid(), signal.SIGTERM)
else:
  print('Kernel is valgrind-ready!')

Por comodidad *Replica* esta celda en una pestaña. Este script compila con gcc 

In [ ]:
%%bash
echo "Compilando con gcc ...." 
gcc -o pp pp.c -std=c99

echo "Resultado de la ejecución del fichero $file..."
echo 
# valgrind ./pp
./pp

## Herramientas fundamentales

* Para visualizar la gestión de memoria: [Python Tutor](http://pythontutor.com/visualize.html#mode=edit) es
una herramienta excepcional para entender la gestión de memoria. Por favor prueba en ella los códigos de esta página seleccionando el lenguaje C.En la mayoría de los ejemplos hemos incluído el enlace correspondiente. 

* Un compilador de C: Por ejemplo, [OnelineGDB](https://www.onlinegdb.com/) 
   es un entorno de programación *online* sencillo para compilar, ejecutar y debuggear programas en C. Te permitirá probar los ejercicios de este tutorial pero, probablemente, no los ejercicios del resto del curso.  

* Valgrind 

# Everything You Always Wanted to Know About C Lenguage But Were Afraid to Ask

**Nota** Para ejecutar los códigos puedes hacer copy/paste en  [OnelineGDB](https://www.onlinegdb.com/) excluyendo la sentencia `%%writefile pp.c` que obviamente no es C.

In [ ]:
%%writefile pp.c

#include <stdlib.h>
#include <stdio.h>

int main (int argc, char *argv[]) {
    char *str;
    
    str = "hola mundo";
    fprintf (stdout, "%s", str);
    
    return EXIT_SUCCESS;
}

## Definición de variables   

### ¿Que implica definir una variable en C? 

Supón el siguiente programa en el que definimos `a` como variable de tipo entero y después le asignamos el valor 5 
[Python Tutor](http://pythontutor.com/visualize.html#code=%23include%20%3Cstdlib.h%3E%0A%23include%20%3Cstdio.h%3E%0A%0Aint%20main%20%28int%20argc,%20char%20*argv%5B%5D%29%20%7B%0A%20%20%20%20int%20a%3B%20%20%20%20//%20definici%C3%B3n%20y%20declaraci%C3%B3n%0A%20%20%20%20a%20%3D%205%3B%20%20%20%20//%20asignaci%C3%B3n%0A%20%20%20%20%0A%20%20%20%20fprintf%20%28stdout,%20%22El%20valor%20de%20a%20es%3A%20%25d%5Cn%22,%20a%29%3B%0A%20%20%20%20fprintf%20%28stdout,%20%22La%20direcci%C3%B3n%20de%20a%20es%3A%20%25p%5Cn%22,%20%26a%29%3B%0A%20%20%20%20%0A%20%20%20%20return%20EXIT_SUCCESS%3B%0A%7D&cumulative=false&curInstr=0&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=c&rawInputLstJSON=%5B%5D&textReferences=false)

In [ ]:
%%writefile pp.c

#include <stdlib.h>
#include <stdio.h>


int main () {
    int a;    // definición y declaración
    char c;
    a = 5;    // asignación
    
    fprintf (stdout, "El valor de a es: %d\n", a);
    fprintf (stdout, "La dirección de a es: %p\n", &a);
    
    return EXIT_SUCCESS;
}

A diferencia de otros lenguajes de programación, en C es necesario definir el tipo de la variables antes de asignarles un valor. 
Cuando definimos una variable en C:

1. Se reserva espacio *físico* en memoria para la variable:

	* El espacio tendrá un tamaño adecuado para el tipo de dato que se almacenará en la variable, en este caso un entero. Puedes obtener el tamaño de cualquier tipo de dato con el operador `sizeof`:

    * El espacio reservado tendrá una dirección. **NO** podemos modificar la dirección de un variable pero sí podemos preguntar cúal es.

    * La variable sólo es válida (existe) en el bloque de código donde se declaró. Recuerda que en C un bloque de código está delimitado por los símbolos de apertura  `{` y cierre `}`. [Python Tutor](http://pythontutor.com/visualize.html#code=%23%20include%20%3Cstdio.h%3E%0A%23%20include%20%3Cstdlib.h%3E%0A%20%0Aint%20main%20%28int%20argc,%20char%20*argv%5B%5D%29%20%7B%0A%20%20int%20i%20%3D%201%3B%0A%20%20char%20c%20%3D'b'%3B%0A%20%20long%20int%20l%20%3D%20150000%3B%0A%20%0A%20%20fprintf%20%28stdout,%20%22%25ld%20%25ld%20%25ld%5Cn%22,%20sizeof%28c%29,%20sizeof%28i%29,%20sizeof%28l%29%20%29%3B%0A%20%20fprintf%20%28stdout,%20%22%25ld%20%25ld%20%25ld%5Cn%22,%20sizeof%28char%29,%20sizeof%28int%29,%20sizeof%28long%20int%29%29%3B%0A%20%0A%20%20//%20address%20memory%0A%20%20fprintf%20%28stdout,%20%22%25p%20%25p%20%25p%5Cn%22,%20%26c,%20%26i,%20%26l%29%3B%0A%20%20//%20Values%0A%20%20fprintf%20%28stdout,%20%22%25c%20%25d%20%25ld%5Cn%22,%20c,%20i,%20l%29%3B%0A%0A%20%20return%20EXIT_SUCCESS%3B%0A%7D&cumulative=false&curInstr=9&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=c&rawInputLstJSON=%5B%5D&textReferences=false)

```c
# include <stdio.h>
# include <stdlib.h>
 
int main (int argc, char *argv[]) {
  int i = 1;
  char c ='b';
  long int l = 150000;
 
  fprintf (stdout, "%ld %ld %ld\n", sizeof(c), sizeof(i), sizeof(l) );
  fprintf (stdout, "%ld %ld %ld\n", sizeof(char), sizeof(int), sizeof(long int));
 
  // address memory
  fprintf (stdout, "%p %p %p\n", &c, &i, &l);
  // Values
  fprintf (stdout, "%c %d %ld\n", c, i, l);

  return EXIT_SUCCESS;
}
```
2.  En C una variable es un nombre que identifica una (o varias) posiciones de memoria donde guardaremos un dato. 

3. Abusando del lenguaje, es útil *pensar* que cuando definimos una variable *creamos una entidad (u objeto) de un determinado **tipo** al que le damos un nombre*. En nuestro ejemplo `a` será un objeto (o entidad) capaz de almacenar un dato de tipo entero. También oirás decir que `a` es un *objeto de tipo entero* o simplemente que `a` es un entero. 

4. Al declarar una variable de un determinado tipo, estamos permitiendo que la variable utilice un conjunto de **operadores**. Los operadores concretos que pueden *operar* sobre una variable dependerán del tipo de la variable. Por ejemplo, el conjunto de operadores que pueden operar sobre de las variables de tipo `int` es distinto al de las variable de tipo  `char`, o al de las variable de tipo *puntero a entero*.
  
 Ejemplos de operadores de las variables de tipo `int` son:

* El operador de asignación `=` : Por ejemplo `a = 5 `. **Importante:** En una asignación los dos miembros de la asignación deben de ser del mismo tipo. En este caso `a` es de tipo entero y `3` es un entero, por tanto la asignación es correcta. 
	
* Los operadores aritméticos como el operador de adición `+`: `a = a + 3` o el de multiplicación `a*3`
	
* El operador de incremento y asignación: `a++`. Esta sentencia es equivalente a `a = a + 1`	
    
* El operador `&` que devuelve la dirección de memoria de la variable: `fprintf (stdout, "La dirección de a es: %p", &a); `  
 
* El operador `sizeof` que devuelve el tamaño de la variable

Ejemplos de operadores de las variables de tipo `char` son:

* El operador de asignación : `a = '3'`. Notad que las comillas son simples **no** dobles.  
    
* ¿Está definido el operador de adición para los objetos de tipo `char`? Pruébalo con el siguiente código:	



In [ ]:
%%writefile pp.c

#include <stdlib.h>
#include <stdio.h>

int main (int argc, char *argv[]) {
    char a = '0';
    
    fprintf (stdout, "El valor a es: %c\n", a);
    
    a = a + 1;
    fprintf (stdout,"El valor de a es: %c\n", a);
    
    a = a + '1';
    fprintf (stdout,"El valor de a es: %c\n", a);

    return EXIT_SUCCESS;
}

## Algunos tipos de dato:



### Puntero simple

Supongamos la siguiente declaración: 

```c
 int *pa;
```

Acostúmbrate a leer la definición anterior de derecha a izquierda. El asterisco indica que `pa` es una variable (o una entidad, o un objeto como prefieras llamarlo) a la que *asignaremos* (en la que *vamos a guardar*) la dirección de un objeto. En este caso en `pa` podremos guardar la dirección un entero.

Algunos operadores accesibles a los objetos de tipo puntero son:

* Asignación `=`: Al haber definido `pa` como un objeto que *guarda* direcciones de enteros *solo* deberíamos asignarle direcciones de objetos que almacenen enteros. 
¿Son correctos estos ejemplos?:

```c
 char *pc;
 char c;
 int a;

 pc = &c;
 pc = &a; //  Warninig
```
* Direccionamiento `&`: Como cualquier objeto en `C`, un puntero también tendrá una dirección. No confundas la dirección del puntero con la dirección que guardamos en el puntero ("a la que apunta" el puntero). Podemos acceder a la dirección del puntero con el operador `&`. Por ejemplo `printf("%p" &p)`. 
* Desreferenciar `*`: Este operador nos permitirá acceder al contenido de la dirección guardada el puntero. Obviamente antes de *desreferenciar* un puntero debemos haber guardado previamente en él una dirección válida (*debemos haberle asignado previamente una dirección válida*) 

````c
int *pa;
int a;
pa = &a;
*pa = 5;
````
* ¿Tienen sentido las siguientes sentencias?¿Son correctas las sentencias comentadas? [Python Tutor]( http://pythontutor.com/visualize.html#code=%0A%0A%23include%20%3Cstdlib.h%3E%0A%23include%20%3Cstdio.h%3E%0A%0Aint%20main%20%28int%20argc,%20char%20*argv%5B%5D%29%20%7B%0A%20%20%20%20int%20*pa%20%3D%20NULL%3B%0A%20%20%20%20int%20a%20%3B%0A%20%20%20%20%0A%20%20%20%20fprintf%20%28stdout,%20%22%25p%20%25p%5Cn%22,%20pa,%20%26a%29%3B%0A%20%20%20%20%0A%20%20%20%20pa%20%3D%20%26a%3B%20%20//%20pa%20espera%20que%20le%20asignemos%20la%20direcci%C3%B3n%20de%20una%20entidad%20que%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20//%20almacena%20un%20entero.%20Como%20la%20variable%20%60a%60%20guarda%20un%20entero%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20//%20la%20asignaci%C3%B3%C3%B3n%20es%20v%C3%A1lida.%20%0A%20%20%20%20%0A%20%20%20%20fprintf%20%28stdout,%20%22%25p%20%25p%5Cn%22,%20pa,%20%26a%29%3B%0A%20%20%20%20%0A%20%20%20%20*pa%20%3D%2025%3B%20%20%20//Desreferenciar%3A%20Accedo%20al%20contenido%20de%20la%20direcci%C3%B3n%20guardada%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20//%20en%20pa%0A%20%20%20%20%0A%20%20%20%20%20%20fprintf%20%28stdout,%20%22%25d,%20%25d%20%5Cn%22,%20a,%20*pa%29%3B%20%0A//%20%20%20%20fprintf%20%28stdout,%20%22%25d%20%5Cn%22,%20*a%29%3B%0A%20%20%20%20%20%20fprintf%20%28stdout,%20%22%25d%20%5Cn%22,%20*%26a%29%3B%0A%20%20%20%20%0A%20%20%20%20return%20EXIT_SUCCESS%3B%0A%7D&cumulative=false&curInstr=9&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=c&rawInputLstJSON=%5B%5D&textReferences=false)

In [ ]:
%%writefile pp.c

#include <stdlib.h>
#include <stdio.h>

int main (int argc, char *argv[]) {
    int *pa = NULL;
    int a ;
    
    fprintf (stdout, "%p %p\n", pa, &a);
    
    pa = &a;  // pa espera que le asignemos la dirección de una entidad que 
              // almacena un entero. Como la variable `a` guarda un entero 
              // la asignacióón es válida. 
    
    fprintf (stdout, "%p %p\n", pa, &a);
    
    *pa = 25;   //Desreferenciar: Accedo al contenido de la dirección guardada 
                // en pa
    
      fprintf (stdout, "%d, %d \n", a, *pa); 
//    fprintf (stdout, "%d \n", *a);
     fprintf (stdout, "%d \n", *(&a) );
    
    return EXIT_SUCCESS;
}
  

* y este otro código ¿Son correctas las sentencias comentadas? [Python Tutor](http://pythontutor.com/visualize.html#code=%0A%23include%20%3Cstdlib.h%3E%0A%23include%20%3Cstdio.h%3E%0A%0Aint%20main%20%28int%20argc,%20char%20*argv%5B%5D%29%20%7B%0A%20%20%20%20int%20*pa%3B%0A%20%20%20%20int%20a%20%3D%203%3B%0A%20%20%20%20%0A%20%20%20%20pa%20%3D%20%26a%3B%20%20%0A%20%20%20%20%0A%20%20%20%20*pa%20%3D%2025%3B%0A%20%20%20%20%0A%20%20%20%20fprintf%20%28stdout,%20%22%25d,%20%25d,%20%25d%20%5Cn%22,%20a,%20*pa,%20*pa%20%3D%3D%20a%29%3B%20%0A//%20%20%20%20fprintf%20%28stdout,%20%22%25d%20%5Cn%22,%20*a%29%3B%0A//%20%20%20%20fprintf%20%28stdout,%20%22%25d%20%5Cn%22,%20*%26a%29%3B%0A%20%20%20%20fprintf%20%28stdout,%20%22%26a%20%3D%20%25p%5Cnpa%20%3D%20%25p,%20%25d%5Cn%26pa%20%3D%20%25p%20%25d%5Cn%22,%20%26a,%20pa,%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%26a%20%3D%3D%20pa,%20%26pa,%20pa%20%3D%3D%20%26pa%29%3B%0A//%20%20%20%20%26a%20%3D%20pa%3B%0A%0A%20%20%20%20return%20EXIT_SUCCESS%3B%0A%7D&cumulative=false&curInstr=6&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=c&rawInputLstJSON=%5B%5D&textReferences=false)



In [ ]:
%%writefile pp.c

#include <stdlib.h>
#include <stdio.h>

int main (int argc, char *argv[]) {
    int *pa;
    int a = 3;
    
    pa = &a;  
    
    *pa = 25;
    
    fprintf (stdout, "%d, %d, %d \n", a, *pa, *pa == a); 
//    fprintf (stdout, "%d \n", *a);
//    fprintf (stdout, "%d \n", *&a);
    fprintf (stdout, "&a = %p\npa = %p, %d\n&pa = %p %d\n", &a, pa, 
             &a == pa, &pa, pa == &pa);
//    &a = pa;

    return EXIT_SUCCESS;
}
   

* y este otro código ¿Son correctas las sentencias comentadas? [Python tutor](http://pythontutor.com/visualize.html#code=%0A%23include%20%3Cstdlib.h%3E%0A%23include%20%3Cstdio.h%3E%0A%0Aint%20main%20%28int%20argc,%20char%20*argv%5B%5D%29%20%7B%0A%20%20%20%20int%20*pa,%20*pb%3B%0A%20%20%20%20int%20a%20%3D%203%3B%0A%20%20%20%20%0A//%20%20%20%20*pa%20%3D%205%3B%0A%20%20%20%20a%2B%2B%3B%20%20%0A%0A%20%20%20%20pa%20%3D%20%26a%3B%0A%20%20%20%20pb%20%3D%20pa%3B%0A%20%20%20%20%0A%20%20%20%20fprintf%20%28stdout,%20%22%25d%20%25d%20%25d%5Cn%22,%20a,%20*pa,%20*pb%29%3B%0A%20%20%20%20fprintf%20%28stdout,%20%22pa%20%3D%20%25p%20%5Cnpb%20%3D%20%25p%22,%20pa,%20pb%29%3B%0A%0A//%20%20%20%20%20pa%20%3D%20%26pb%3B%0A//%20%20%20%20%20%26pb%20%3D%20%26pa%3B%0A//%20%20%20%20%20%26a%20%3D%20pa%3B%20%20%20%20%0A%20%20%20%20%0A%20%20%20%20%20return%20EXIT_SUCCESS%3B%0A%7D&cumulative=false&curInstr=0&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=c&rawInputLstJSON=%5B%5D&textReferences=false)

In [ ]:
%%writefile pp.c

#include <stdlib.h>
#include <stdio.h>

int main (int argc, char *argv[]) {
    int *pa, *pb;
    int a = 3;
    
//    *pa = 5;
    a++;  

    pa = &a;
    pb = pa;
    
    fprintf (stdout, "%d %d %d\n", a, *pa, *pb);
    fprintf (stdout, "pa = %p \npb = %p", pa, pb);

//     pa = &pb;
//     &pb = &pa;
//     &a = pa;    
    
     return EXIT_SUCCESS;
}
   

###  Doble puntero


```c
	int **ppa;
```

¿Cómo interpretas la definición anterior? ¿Qué significa definir `ppa` como un *doble puntero*? 
Recuerda que estás en C y en C todo tiene un significado muy preciso. Nunca intentes memorizar una expresión sino entenderla.

Como siempre debes leer la definición `int **ppa` de derecha a izquierda. El asterisco derecho nos indica que en `ppa` vamos a guardar la dirección de un objeto. ¿De qué *tipo* de objeto? Como el tipo de objeto lo indica todo lo que se encuentra a la izquierda del primer asterisco (contando por la derecha), en este caso en el puntero `ppa` debemos guardar  (asignarle) la dirección de un objeto de tipo *puntero a entero* (a la izquierda de `*` tenemos `int *`). Es decir de un objeto capaz de guardar la dirección de un entero.

```c
int a;   // objeto que guardará un entero
int *pa;   // objeto que guardará la dirección de un entero
int **ppa; // objeto que guardará la dirección 
           // de un objeto que guardará la dirección de un entero

a = 3;
pa = &a;
ppa = &pa; // Al objeto ppa le asignamos la direccióón de pa 
           // siendo pa es un punetro a entero
````

* ¿Tienen sentido las siguientes sentencias?¿Es correcto el orden en que aparecen? Revisa los dibujos correspondientes a la memoria para entenderlas. [Python tutorial](http://pythontutor.com/visualize.html#code=%0A%23include%20%3Cstdlib.h%3E%0A%23include%20%3Cstdio.h%3E%0A%0Aint%20main%20%28int%20argc,%20char%20*argv%5B%5D%29%20%7B%0A%20%20%20%20int%20**ppa%20%3D%20NULL%3B%0A%20%20%20%20int%20*pa%20%3D%20NULL%3B%0A%20%20%20%20int%20a%3B%0A%20%20%20%20%0A%20%20%20%20ppa%20%3D%20%26pa%3B%0A%20%20%20%20fprintf%20%28stdout,%20%22%25p%20%20%25p%20%5Cn%22,%20ppa,%20%26pa%29%3B%0A%0A%20%20%20%20pa%20%3D%20%26a%3B%0A%20%20%20%20fprintf%20%28stdout,%20%22%25p%20%20%25p%20%5Cn%22,%20pa,%20%26a%29%3B%0A%20%20%20%20%0A%20%20%20%20a%20%3D%205%3B%0A%20%20%20%20%0A%20%20%20%20fprintf%20%28stdout,%20%22%25d%20%5Cn%22,%20a%29%3B%0A%20%20%20%20fprintf%20%28stdout,%20%22%25d%20%5Cn%22,%20*pa%29%3B%0A%20%20%20%20fprintf%20%28stdout,%20%22%25d%20%5Cn%22,%20**ppa%29%3B%0A%20%20%20%20fprintf%20%28stdout,%20%22%25p%20%25p%20%5Cn%22,%20pa,%20%26a%29%3B%0A%20%20%20%20fprintf%20%28stdout,%20%22%25p%20%25p%20%25p%22,%20%26ppa,%20ppa,%20%26pa%29%3B%0A%20%20%20%20%0A%20%20%20%20return%20EXIT_SUCCESS%3B%0A%20%20%20%20%0A%7D&cumulative=false&curInstr=12&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=c&rawInputLstJSON=%5B%5D&textReferences=false)

In [ ]:
%%writefile pp.c

#include <stdlib.h>
#include <stdio.h>

int main (int argc, char *argv[]) {
    int **ppa = NULL;
    int *pa = NULL;
    int a;
    
    ppa = &pa;
    fprintf (stdout, "%p  %p \n", ppa, &pa);

    pa = &a;
    fprintf (stdout, "%p  %p \n", pa, &a);
    
    a = 5;
    
    fprintf (stdout, "%d \n", a);
    fprintf (stdout, "%d \n", *pa);
    fprintf (stdout, "%d \n", **ppa);
    fprintf (stdout, "%p %p \n", pa, &a);
    fprintf (stdout, "%p %p %p", &ppa, ppa, &pa);
    
    return EXIT_SUCCESS;
    
}

###  Arrays


```c
	int a[3];
```

Con esta declaración estamos indicando que se reserve **memoria secuencial estática** (contigua) para tres objetos. Cada uno de los objetos deberá ser del tipo que tengamos a la izquierda del nombre del array, `a`. En este caso hemos definido una cantidad de memoria suficiente para almacenar *tres objetos de tipo entero*. O con mas precisión: memoria suficiente para tres objetos cada uno con un tamaño suficiente para guardar un entero. 

Algunas cuestiones importantes que debemos tener en cuenta:

* El nombre del array, `a`, indica la dirección del primero de los tres objetos que componen el array.	

* Se puede acceder a los valores almacenados en el array con los corchetes. Con `a[0]` accedemos al valor del entero guardado en el primero de los tres objetos que componen el array.

* También se puede acceder a la dirección de cada uno de los objetos del array. Hay dos formas de hacerlo:

	* Con el operador de dirección `&`: Por ejemplo: con  `&a[0]` accederíamos a la dirección del primer elemento del array. La expresión anterior debes interpretarla como `&(a[0])` y **no** como `(&a)[0]` (recuerda que el nombre del array es una dirección) . Por tanto `&a[0]`es equivalente a `a` y la comparación lógica `&a[0] == a` se evalúa como cierta.

	* Utilizando la *aritmética de punteros*: Como `a` es la dirección del primer elemento del array, `a+1` es la dirección del segundo de los elementos del array. Por tanto `a+1 == &a[1]` es cierto.

¿Tienen sentido las siguientes sentencias? Haz los dibujos de memoria para entenderlas.


In [ ]:
%%writefile pp.c

#include <stdlib.h>
#include <stdio.h>

int main (int argc, char *argv[]) {
    int a[] = {0, 1, 2};
    int *pa;
    int size, i;
    
    size = sizeof(a) / sizeof (int);
    
    a[0]++;      // equivalente a: a[0] = a[0] + 1
    pa = a - 1; 
    
    *(pa + 1) = 77;
//    pa = &a ;
  
    for (i = 0; i < size; i++)
        fprintf (stdout, "%d , %d\n", i, a[i]);
    
    return EXIT_SUCCESS;
}

* Mezclando la sintáxis de punteros y arrays: [Python Tutorial](http://pythontutor.com/visualize.html#code=%0A%23include%20%3Cstdlib.h%3E%0A%23include%20%3Cstdio.h%3E%0A%0Aint%20main%20%28int%20argc,%20char%20*argv%5B%5D%29%20%7B%0A%20%20%20%20int%20a%5B3%5D%3B%0A%20%20%20%20int%20*pa%3B%0A%20%20%20%20int%20i%3B%0A%20%20%20%20%0A%20%20%20%20for%20%28i%20%3D%200%3B%20i%20%3C%203%3B%20i%2B%2B%29%20%7B%0A%20%20%20%20%20%20%20%20a%5Bi%5D%20%3D%20i%20*%20100%3B%0A%20%20%20%20%7D%0A%20%20%20%20%0A%20%20%20%20//%20Asigno%20al%20puntero%20pa%20la%20direcci%C3%B3n%20del%20primer%20elemento%20del%20array%0A%20%20%20%20pa%20%3D%20a%3B%0A%20%20%20%20%0A%20%20%20%20for%20%28i%20%3D%200%3B%20i%20%3C%203%3B%20i%2B%2B%29%20%7B%0A%20%20%20%20%20%20%20%20fprintf%20%28stdout,%20%22%25d%20%20%22,%20a%5Bi%5D%29%3B%0A%20%20%20%20%7D%0A%20%20%20%20%0A%20%20%20%20puts%28%22%5CnArray%20con%20formalismo%20de%20puntero%22%29%3B%0A%20%20%20%20for%20%28i%20%3D%200%3B%20i%20%3C%203%3B%20i%2B%2B%29%20%7B%0A%20%20%20%20%20%20%20%20fprintf%20%28stdout,%20%22%25d%20%20%22,%20*%28a%20%2B%20i%29%29%3B%0A%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20puts%28%22%5Cn%20Accediendo%20al%20puntero%20con%20formalismo%20de%20puntero%22%29%3B%0A%20%20%20%20for%20%28i%20%3D%200%3B%20i%20%3C%203%3B%20i%2B%2B%29%20%7B%0A%20%20%20%20%20%20%20%20fprintf%20%28stdout,%20%22%25d%20%20%22,%20*%28pa%20%2B%20i%29%29%3B%0A%20%20%20%20%7D%20%20%20%20%0A%20%20%20%20%0A%20%20%20%20puts%28%22%5Cn%20Puntero%20con%20formalismo%20de%20array%22%29%3B%0A%20%20%20%20for%20%28i%20%3D%200%3B%20i%20%3C%203%3B%20i%2B%2B%29%20%7B%0A%20%20%20%20%20%20%20%20fprintf%20%28stdout,%20%22%25d%20%20%22,%20pa%5Bi%5D%29%3B%0A%20%20%20%20%7D%20%20%20%0A%20%20%20%20%0A/*%0A%20%20%20%20//%20Es%20lo%20que%20quieres%3F%0A%20%20%20%20puts%28%22%5Cn%22%29%3B%0A%20%20%20%20for%20%28i%20%3D%200%3B%20i%20%3C%203%3B%20i%2B%2B%29%20%7B%0A%20%20%20%20%20%20%20%20fprintf%20%28stdout,%20%22%25d%20%20%22,%20%28*pa%20%2B%20i%29%20%29%3B%0A%20%20%20%20%7D%20%20%20%0A*/%0A%20%20%20%20%0A%20%20%20%20return%20EXIT_SUCCESS%3B%0A%7D&cumulative=false&curInstr=0&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=c&rawInputLstJSON=%5B%5D&textReferences=false)

In [ ]:
%%writefile pp.c

#include <stdlib.h>
#include <stdio.h>

int main (int argc, char *argv[]) {
    int a[3];
    int *pa;
    int i;
    
    for (i = 0; i < 3; i++) {
        a[i] = i * 100;
    }
    
    // Asigno al puntero pa la dirección del primer elemento del array
    pa = a;
    
    for (i = 0; i < 3; i++) {
        fprintf (stdout, "%d  ", a[i]);
    }
    
    puts("\nArray con formalismo de puntero");
    for (i = 0; i < 3; i++) {
        fprintf (stdout, "%d  ", *(a + i));
    }
                 
    puts("\n Accediendo al puntero con formalismo de puntero");
    for (i = 0; i < 3; i++) {
        fprintf (stdout, "%d  ", *(pa + i));
    }    
    
    puts("\n Accediendo al puntero con formalismo de array");
    for (i = 0; i < 3; i++) {
        fprintf (stdout, "%d  ", pa[i]);
    }   
    
/*
    // Es lo que quieres?
    puts("\n");
    for (i = 0; i < 3; i++) {
        fprintf (stdout, "%d  ", (*pa + i) );
    }   
*/
    
    return EXIT_SUCCESS;
}

### Arrays de punteros


```c
	int *a[3];
```

La definición anterior no te debería sorprender si la les de derecha a izquierda: Estamos declarando que `a` es un array de tres objetos y que en cada uno de los objetos guardaremos la dirección de un entero. Es decir, cada uno de los objetos del array es un puntero a entero.

¿Es correcto el siguiente programa? ¿Cuál sería su salida? Ejecútalo en [Python-tutor](http://pythontutor.com/visualize.html#code=%0A%23include%20%3Cstdlib.h%3E%0A%23include%20%3Cstdio.h%3E%0A%0Aint%20main%20%28int%20argc,%20char%20*argv%5B%5D%29%20%7B%0A%20%20%20%20int%20a%5B3%5D%3B%0A%20%20%20%20int%20*pa%5B3%5D%3B%0A%20%20%20%20int%20i%3B%0A%0A%20%20%20%20for%20%28i%20%3D%200%3B%20i%20%3C%203%3B%20i%2B%2B%29%7B%0A%20%20%20%20%20%20%20%20pa%5Bi%5D%20%3D%20a%20%2B%20i%3B%0A%20%20%20%20%7D%0A%20%20%20%20%0A%20%20%20%20for%20%28i%20%3D%200%3B%20i%20%3C%203%3B%20i%2B%2B%29%7B%0A%20%20%20%20%20%20%20%20*%28*%28pa%20%2B%20i%29%29%20%3D%20100%20*%20i%3B%0A%20%20%20%20%7D%0A%0A%20%20%20%20for%20%28i%20%3D%200%3B%20i%20%3C%203%3B%20i%2B%2B%29%0A%20%20%20%20%20%20%20%20fprintf%20%28stdout,%20%22%25d%20%7C%20%25d%20%20%5Cn%22,%20a%5Bi%5D,%20*pa%5Bi%5D%29%3B%0A%20%20%20%20%0A%20%20%20%20return%20EXIT_SUCCESS%3B%0A%7D%0A%20%20%0A&cumulative=false&curInstr=21&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=c&rawInputLstJSON=%5B%5D&textReferences=false)


In [ ]:
%%writefile pp.c

#include <stdlib.h>
#include <stdio.h>

int main (int argc, char *argv[]) {
    int a[3];
    int *pa[3];
    int i;

    for (i = 0; i < 3; i++){
        pa[i] = a + i;
    }
    
    for (i = 0; i < 3; i++){
        *(*(pa + i)) = 100 * i;
    }

    for (i = 0; i < 3; i++)
        fprintf (stdout, "%d | %d  \n", a[i], *pa[i]);
    
    return EXIT_SUCCESS;
}
  


### Arrays de dos dimensiones

```c
int a[5][3]`
````
En una declaración como la anterior tenedemos a pensar en el objeto `a` como una matriz de de 5 filas y 3 columnas. Sin embargo hay que tener cuidado con esta interpretación. La declaración debemos entenderla como un array de 5 objetos donde cada objeto del array es, a su vez, un array de tres objetos de tipo entero. 
Si observas la salida del programa siguiente verás como la memoria que se ha reservado es secuencial (contigua). La direccióón de memoria de cada casilla del array puede obtenerse a partir de la direccióón de la memoria de la casilla anterior del array sumándole el tamaño en Bytes de un entero `sizeof(int) = 4`.   


In [ ]:
%%writefile pp.c

#include <stdio.h>
#include <stdlib.h>

#define num1 5
#define num2 3

int main()
{
    int x[num1][num2];
    int i, j;
    
    // Assign values
    for (i=0; i < num1 ; i++) {
        for (j=0; j < num2; j++) {
            x[i][j] = 100 * i + j ;
        }
    }
    
    // Note how the memory is sequential!!!
    for (i=0; i < num1 ; i++) 
        for (j=0; j < num2; j++) 
            fprintf (stdout, "(%d,%d):%p \n", i, j, &(x[i][j]));
    
    return EXIT_SUCCESS;
}


## Funciones

* El siguiente programa asigna valores a las variables en el mismo bloque de código donde fueron definidas

```c
	#include <stdio.h>

	main () {
		int a;
        int b = 5;
        int ar[] = {1, 2, 3};
        int *pa = NULL;

		a = 3;   // asignación dentro del bloque de código donde fue declarada
        ar[0] = 55;
        pa = ar + 1;
	}

```
* Las funciones nos permiten asignar valor a una variable fuera del bloque de código donde fue definida. En el siguiente programa a la variable  `a` le asignaremos un valor, primero con la función `fasg1` y posteriormente con la función `fasg2`  [Python tutor](http://pythontutor.com/visualize.html#code=%0A%23include%20%3Cstdlib.h%3E%0A%23include%20%3Cstdio.h%3E%0A%0Aint%20fasg1%20%28%29%3B%20%20%0Avoid%20fasg2%20%28int%20*%29%3B%0A%0Aint%20main%20%28int%20argc,%20char%20*argv%5B%5D%29%20%7B%0A%20%20%20%20%0A%20%20%20%20int%20a%20%3D%203%3B%0A%0A%20%20%20%20fprintf%20%28stdout,%20%22Antes%20de%20llamada%20a%20funciones%3A%20%25d%5Cn%22,%20a%29%3B%0A%0A%20%20%20%20a%20%3D%20fasg1%20%28%29%3B%20%20//%20Por%20retorrno%20de%20la%20funcion%0A%20%20%20%20fprintf%20%28stdout,%20%22Despues%20de%20asignacion%201%3A%20%25d%5Cn%22,%20a%29%3B%0A%0A%20%20%20%20fasg2%20%28%26a%29%3B%20%20//%20%0A%20%20%20%20fprintf%20%28stdout,%20%22Despues%20de%20asignacion%202%3A%20%25d%5Cn%22,%20a%29%3B%0A%0A%20%20%20%20return%20EXIT_SUCCESS%3B%0A%7D%0A%0A//%20Definicion%20funcion%0Aint%20fasg1%20%28%29%20%7B%0A%20%20%20%20int%20b%3B%0A%0A%20%20%20%20b%20%3D%20100%3B%0A%20%20%20%20return%20b%3B%0A%7D%0A%0Avoid%20fasg2%20%28int%20*a%29%20%7B%0A%20%20%20%20*a%20%3D%20200%3B%0A%0A%20%20%20%20return%3B%0A%7D&cumulative=false&curInstr=13&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=c&rawInputLstJSON=%5B%5D&textReferences=false)




In [ ]:
%%writefile pp.c

#include <stdlib.h>
#include <stdio.h>

int fasg1 ();  
void fasg2 (int *);

int main (int argc, char *argv[]) {
    
    int a = 3;

    fprintf (stdout, "Antes de llamada a funciones: %d\n", a);

    a = fasg1 ();  // Por retorrno de la funcion
    fprintf (stdout, "Despues de asignacion 1: %d\n", a);

    fasg2 (&a);  // 
    fprintf (stdout, "Despues de asignacion 2: %d\n", a);

    return EXIT_SUCCESS;
}

// Definicion funcion
int fasg1 () {
    int b;

    b = 100;
    return b;
}

void fasg2 (int *a) {
    *a = 200;

    return;
}

* Asignar valor a un **array** fuera del bloque de código donde fue definido [Python-tutor](http://pythontutor.com/visualize.html#code=%0A%23include%20%3Cstdlib.h%3E%0A%23include%20%3Cstdio.h%3E%0A%0Avoid%20fasg%20%28int%20*,%20int%29%3B%20%0A%0Aint%20main%20%28int%20argc,%20char%20*argv%5B%5D%29%20%7B%0A%20%20%20%20%0A%20%20%20%20int%20a%5B%5D%20%3D%20%7B1,%202,%203%7D%3B%0A%20%20%20%20int%20i,%20size%3B%0A%0A%20%20%20%20//%20Obtengo%20el%20tama%C3%B1o%20del%20array%20%28size%20%3D%203%29%0A%20%20%20%20size%20%3D%20sizeof%28a%29%20/%20sizeof%20%28int%29%3B%0A%0A%20%20%20%20puts%20%28%22%5Cn%20Antes%20de%20las%20funciones%22%29%3B%0A%20%20%20%20for%20%28i%20%3D%200%3B%20i%20%3C%20size%3B%20i%20%2B%2B%29%20%7B%0A%20%20%20%20%20%20fprintf%20%28stdout,%20%22%25d%20%22,%20a%5Bi%5D%29%3B%0A%20%20%20%20%7D%0A%0A%20%20%20%20fasg%20%28a,%20size%29%3B%20%20//%20Paso%20una%20direcci%C3%B3%C3%B3n%20-%3E%20la%20direcci%C3%B3n%20del%20primer%20objeto%20del%20array%0A%20%20%20%20%0A%20%20%20%20puts%20%28%22%5Cn%20Despues%20de%20las%20funciones%22%29%3B%0A%20%20%20%20for%20%28i%20%3D%200%3B%20i%20%3C%20size%3B%20i%20%2B%2B%29%20%7B%0A%20%20%20%20%20%20fprintf%20%28stdout,%20%22%25d%20%22,%20a%5Bi%5D%29%3B%0A%20%20%20%20%7D%0A%0A%20%20%20%20return%20EXIT_SUCCESS%3B%0A%7D%0A%0Avoid%20fasg%20%28int%20*b,%20int%20size%29%20%7B%0A%20%20%20%20int%20i%3B%0A%0A%20%20%20%20for%20%28i%20%3D%200%3B%20i%20%3C%20size%3B%20i%20%2B%2B%29%20%7B%0A%20%20%20%20%20%20%20%20b%5Bi%5D%20%3D%20100%20*%20i%3B%0A%20%20%20%20%7D%0A%20%20%20%20%0A%20%20%20%20return%3B%0A%7D%0A%0A&cumulative=false&curInstr=22&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=c&rawInputLstJSON=%5B%5D&textReferences=false)





In [ ]:
%%writefile pp.c

#include <stdlib.h>
#include <stdio.h>

void fasg (int *, int); 

int main (int argc, char *argv[]) {
    
    int a[] = {1, 2, 3};
    int i, size;

    // Obtengo el tamaño del array (size = 3)
    size = sizeof(a) / sizeof (int);

    puts ("\n Antes de las funciones");
    for (i = 0; i < size; i ++) {
      fprintf (stdout, "%d ", a[i]);
    }

    fasg (a, size);  // Paso una direccióón -> la dirección del primer objeto del array
    
    puts ("\n Despues de las funciones");
    for (i = 0; i < size; i ++) {
      fprintf (stdout, "%d ", a[i]);
    }

    return EXIT_SUCCESS;
}

void fasg (int *b, int size) {
    int i;

    for (i = 0; i < size; i ++) {
        b[i] = 100 * i;
    }
    
    return;
}



**Analiza:** 

1. Uno de los argumentos de entrada de la función `fasg` del código anterior era el tamaño del array al cúal queríamos asignar valores. ¿Es necesario pasar a la función el tamaño del array o podríamos haberlo calcularlo dentro de la función por ejemplo con el operador `sizeof` o con la función de la biblioteca `strlen`?
 
 ```c
void fasg (int *b) {
    int i;
    int size;

    // size = sizeof (b);
    // size = strlen (b);

    for (i = 0; i < size; i ++) {
        b[i] = 100 * i;
    }
    
    return;
}
```

2. En vez de pasar la dirección del array que queremos modificar como hemos hecho en la función `fasg`  ¿sería posible para asignar nuevos valores al array hacer una función que nos devolviese los valores del array por retorno? Es decir, algo parecido al siguiente código:

 ```c
int main (int argc, char *argv[]) {
    
    int a[] = {1, 2, 3};
    int i, size;

    // Obtengo el tamaño del array (size = 3)
    size = sizeof(a) / sizeof (int);

    puts ("\n Antes de las funciones");
    for (i = 0; i < size; i ++) {
      fprintf (stdout, "%d ", a[i]);
    }

    a = fasg (...........);  
        :
        :
        : 
}
```
**IMPORTANTE:** 

Para asignar un valor a un objeto fuera del bloque del que ha sido declarado es necesario crear una función. 
Como regla general, en los parámetros de la función *debemos pasar* la dirección del objeto del cual queremos modificar su valor. Algunos objetos de `C`, como los arrays, necesariamente deben modificarse de esta forma ya que **NO** admiten la modificación por retorno de la función.





**EJERCICIO:** 
Completa la declaración, definición y llmada de la función *fSwapp* del código siguiente

In [ ]:
%%writefile pp.c

#include <stdlib.h>
#include <stdio.h>

// void fSwapp (....., ....); 

int main (int argc, char *argv[]) {
    char c1[] = {'a', 'b', 'c'};
    char c2[] = {'h', 'i', 'k'};
    char *pc1, *pc2;
    int i;

    pc1 = c1;
    pc2 = c2;
    fprintf (stdout, "%p %p \n %p %p", pc1, c1, pc2, c2);

    puts("pc1:");
    for (i=0; i < 3; i++) {
      printf ("%c  ", pc1[i]);
    }

    puts("\npc2:");
    for (i=0; i < 3; i++) {
      printf ("%c  ", pc2[i]);
    }

    /* 
    * Después de ejecutar la función fSwapp quiero que en pc1 se guarde
    * la dirección del array c2  y en pc2 la del array c1
    */

    // fSwapp ( ..., ....);

    puts("\npc1 tras el intercambio:");
    for (i=0; i < 3; i++) {
      printf ("%c  ", pc1[i]);
    }

    puts("\npc2 tras el intercambio:");
    for (i=0; i < 3; i++) {
      printf ("%c  ", pc2[i]);
    } 
    return EXIT_SUCCESS;
}


**Solución:**

[Python Tutor](http://pythontutor.com/visualize.html#code=%23include%20%3Cstdlib.h%3E%0A%23include%20%3Cstdio.h%3E%0A%0A%20void%20fSwapp%20%28char%20**,%20char%20**%29%3B%20%0A%0Aint%20main%20%28int%20argc,%20char%20*argv%5B%5D%29%20%7B%0A%20%20%20%20char%20c1%5B%5D%20%3D%20%7B'a',%20'b',%20'c'%7D%3B%0A%20%20%20%20char%20c2%5B%5D%20%3D%20%7B'h',%20'i',%20'k'%7D%3B%0A%20%20%20%20char%20*pc1%20%3D%20NULL,%20*pc2%20%3D%20NULL%3B%0A%20%20%20%20int%20i%3B%0A%0A%20%20%20%20pc1%20%3D%20c1%3B%0A%20%20%20%20pc2%20%3D%20c2%3B%0A%20%20%20%20puts%28%22Direcciones%5Cn%22%29%3B%0A%20%20%20%20fprintf%20%28stdout,%20%22%25p%20%25p%20%5Cn%25p%20%25p%5Cn%22,%20pc1,%20c1,%20pc2,%20c2%29%3B%0A%0A%20%20%20%20puts%28%22pc1%3A%22%29%3B%0A%20%20%20%20for%20%28i%3D0%3B%20i%20%3C%203%3B%20i%2B%2B%29%20%7B%0A%20%20%20%20%20%20printf%20%28%22%25c%20%20%22,%20pc1%5Bi%5D%29%3B%0A%20%20%20%20%7D%0A%0A%20%20%20%20puts%28%22%5Cnpc2%3A%22%29%3B%0A%20%20%20%20for%20%28i%3D0%3B%20i%20%3C%203%3B%20i%2B%2B%29%20%7B%0A%20%20%20%20%20%20printf%20%28%22%25c%20%20%22,%20pc2%5Bi%5D%29%3B%0A%20%20%20%20%7D%0A%0A%20%20%20%20/*%20%0A%20%20%20%20*%20Despu%C3%A9s%20de%20ejecutar%20la%20funci%C3%B3n%20fSwapp%20quiero%20que%20en%20pc1%20se%20guerde%0A%20%20%20%20*%20la%20direcci%C3%B3n%20del%20array%20c2%20%20y%20en%20pc2%20la%20del%20array%20c1%0A%20%20%20%20*/%0A%0A%20%20%20%20fSwapp%20%28%26pc1,%20%26pc2%29%3B%0A%0A%20%20%20%20puts%28%22%5Cnpc1%20tras%20el%20intercambio%3A%22%29%3B%0A%20%20%20%20for%20%28i%3D0%3B%20i%20%3C%203%3B%20i%2B%2B%29%20%7B%0A%20%20%20%20%20%20printf%20%28%22%25c%20%20%22,%20pc1%5Bi%5D%29%3B%0A%20%20%20%20%7D%0A%0A%20%20%20%20puts%28%22%5Cnpc2%20tras%20el%20intercambio%3A%22%29%3B%0A%20%20%20%20for%20%28i%3D0%3B%20i%20%3C%203%3B%20i%2B%2B%29%20%7B%0A%20%20%20%20%20%20printf%20%28%22%25c%20%20%22,%20pc2%5Bi%5D%29%3B%0A%20%20%20%20%7D%20%0A%20%20%20%20return%20EXIT_SUCCESS%3B%0A%7D%0A%0Avoid%20fSwapp%20%28char%20**p1,%20char%20**p2%29%20%7B%0A%20%20%20%20char%20*buff%3B%0A%0A%20%20%20%20buff%20%3D%20*p1%3B%0A%20%20%20%20*p1%20%3D%20*p2%3B%0A%20%20%20%20*p2%20%3D%20buff%3B%0A%7D&cumulative=false&curInstr=31&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=c_gcc9.3.0&rawInputLstJSON=%5B%5D&textReferences=false)

In [ ]:
%%writefile pp.c

#include <stdlib.h>
#include <stdio.h>

 void fSwapp (char **, char **); 

int main (int argc, char *argv[]) {
    char c1[] = {'a', 'b', 'c'};
    char c2[] = {'h', 'i', 'k'};
    char *pc1 = NULL, *pc2 = NULL;
    int i;

    pc1 = c1;
    pc2 = c2;
    puts("Direcciones\n");
    fprintf (stdout, "%p %p \n%p %p\n", pc1, c1, pc2, c2);

    puts("pc1:");
    for (i=0; i < 3; i++) {
      printf ("%c  ", pc1[i]);
    }

    puts("\npc2:");
    for (i=0; i < 3; i++) {
      printf ("%c  ", pc2[i]);
    }

    /* 
    * Después de ejecutar la función fSwapp quiero que en pc1 se guarde
    * la dirección del array c2  y en pc2 la del array c1
    */

    fSwapp (&pc1, &pc2);

    puts("\npc1 tras el intercambio:");
    for (i=0; i < 3; i++) {
      printf ("%c  ", pc1[i]);
    }

    puts("\npc2 tras el intercambio:");
    for (i=0; i < 3; i++) {
      printf ("%c  ", pc2[i]);
    } 
    return EXIT_SUCCESS;
}

void fSwapp (char **p1, char **p2) {
    char *buff;

    buff = *p1;
    *p1 = *p2;
    *p2 = buff;
}

## Memoria dinámica

Analiza los siguientes códigos. Aplica los conceptos que hemos visto sobre asignaciones de valores a variables fuera del bloque de código donde fueron definidas. Debes convencerte de que lo entiendes **perfectamente**. La lógica de `C` es aplastante. No intentes memorizar reglas mnemotécnicas!!

Verifica tus conclusiones con ayuda de la herramienta Python Tutor.
Debes verificar además que tu programa libera correctamente la memoria reservada. Utiliza para ello la herramienta Valgrind de Linux 

* **Versión 1:**
Se asigna valor a la variable `pa` dentro del bloque de cóódigo donde fue definida. 

  Tanto la asignación como la liberación de la memoria es correcta
[Python tutor](http://pythontutor.com/visualize.html#code=%0A%23include%20%3Cstdlib.h%3E%0A%23include%20%3Cstdio.h%3E%0A%0Aint%20main%20%28int%20argc,%20char%20*argv%5B%5D%29%20%7B%0A%20%20%20%20%0A%20%20%20%20int%20*pa%20%3D%20NULL%3B%0A%0A%20%20%20%20pa%20%3D%20%28int%20*%29%20malloc%20%28sizeof%28int%29%29%3B%0A%20%20%20%20if%20%28pa%20%3D%3D%20NULL%29%20%7B%0A%20%20%20%20%20%20%20%20puts%20%28%22no%20memory%22%29%3B%0A%20%20%20%20%20%20%20%20return%20EXIT_FAILURE%3B%0A%20%20%20%20%7D%0A%0A%20%20%20%20*pa%20%3D%20152%3B%0A%20%20%20%20printf%20%28%22Valor%3A%20%25d%20%20Direcci%C3%B3n%3A%20%25p%22,%20*pa,%20pa%29%3B%0A%0A%20%20%20%20//free%20resources%0A%20%20%20%20free%20%28pa%29%3B%0A%20%20%20%20return%20EXIT_SUCCESS%3B%0A%7D&cumulative=false&curInstr=8&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=c&rawInputLstJSON=%5B%5D&textReferences=false)


In [ ]:
%%writefile pp.c

#include <stdlib.h>
#include <stdio.h>

int main (int argc, char *argv[]) {
    
    int *pa = NULL;

    pa = (int *) malloc (sizeof(int));
    if (pa == NULL) {
        puts ("no memory");
        return EXIT_FAILURE;
    }

    *pa = 152;
    printf ("Valor: %d  Dirección: %p", *pa, pa);

    //free resources
    free (pa);
    return EXIT_SUCCESS;
}

* **Versión 2:** La reserva de memoria la realizamos fuera de la función *main* . 

  La función `f1`  devuelve *por su retorno* la dirección de la memoria reservada [Python tutor](http://pythontutor.com/visualize.html#code=%0A%23include%20%3Cstdlib.h%3E%0A%23include%20%3Cstdio.h%3E%0A%0A%0Aint%20*%20f1%20%28%29%3B%0A%0Aint%20main%20%28int%20argc,%20char%20*argv%5B%5D%29%20%7B%0A%20%20%20%20int%20*pa%20%3D%20NULL%3B%0A%0A%20%20%20%20pa%20%3D%20f1%28%29%3B%0A%20%20%20%20if%20%28!pa%29%0A%20%20%20%20%20%20return%20EXIT_FAILURE%3B%0A%20%20%20%20%0A%20%20%20%20*pa%20%3D%20152%3B%0A%20%20%20%20printf%20%28%22Valor%3A%20%25d%20%20Direcci%C3%B3n%3A%20%25p%22,%20*pa,%20pa%29%3B%0A%0A%20%20%20%20//free%20resources%0A%20%20%20%20free%20%28pa%29%3B%0A%20%20%20%20return%20EXIT_SUCCESS%3B%0A%7D%0A%0Aint%20*f1%20%28%29%20%7B%0A%20%20%20%20int%20*pa%20%3D%20NULL%3B%0A%0A%20%20%20%20pa%20%3D%20%28int*%29%20malloc%20%28sizeof%28int%29%29%3B%0A%20%20%20%20return%20pa%3B%0A%7D%20%0A&cumulative=false&curInstr=11&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=c&rawInputLstJSON=%5B%5D&textReferences=false)

In [ ]:
%%writefile pp.c

#include <stdlib.h>
#include <stdio.h>


int * f1 ();

int main (int argc, char *argv[]) {
    int *pa = NULL;

    pa = f1();
    if (!pa)
      return EXIT_FAILURE;
    
    *pa = 152;
    printf ("Valor: %d  Dirección: %p", *pa, pa);

    //free resources
    free (pa);
    return EXIT_SUCCESS;
}

int *f1 () {
    int *pa = NULL;

    pa = (int*) malloc (sizeof(int));
    return pa;
} 



**Versión 3:**
La reserva de memoria la realizamos fuera de la función main.
La función `f2` *recibe* la dirección del puntero `pa`. La memoria que se reserva en `f2` se le asigna
al contenido de la dirección recibida.  [Python-tutor](http://pythontutor.com/visualize.html#code=%0A%23include%20%3Cstdlib.h%3E%0A%23include%20%3Cstdio.h%3E%0A%0A%0Avoid%20f2%20%28int%20**pa%29%3B%0A%0Aint%20main%20%28int%20argc,%20char%20*argv%5B%5D%29%20%7B%0A%20%20%20%20int%20*pa%20%3D%20NULL%3B%0A%0A%20%20%20%20f2%20%28%26pa%29%3B%0A%20%20%20%20if%20%28!pa%29%20return%20EXIT_FAILURE%3B%0A%0A%20%20%20%20*pa%20%3D%20152%3B%0A%20%20%20%20printf%20%28%22Valor%3A%20%25d%20%20Direcci%C3%B3n%3A%20%25p%22,%20*pa,%20pa%29%3B%0A%0A%20%20%20%20//free%20resources%0A%20%20%20%20free%20%28pa%29%3B%0A%20%20%20%20return%20EXIT_SUCCESS%3B%0A%7D%0A%0A%20void%20f2%20%28int%20**pa%29%20%7B%0A%20%20%0A%20%20%20%20*pa%20%3D%20%28int*%29%20malloc%20%28sizeof%28int%29%29%3B%0A%20%20%20%20return%3B%0A%7D%20%0A&cumulative=false&curInstr=7&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=c&rawInputLstJSON=%5B%5D&textReferences=false)

In [ ]:
%%writefile pp.c

#include <stdlib.h>
#include <stdio.h>


void f2 (int **pa);

int main (int argc, char *argv[]) {
    int *pa = NULL;

    f2 (&pa);
    if (!pa) return EXIT_FAILURE;

    *pa = 152;
    printf ("Valor: %d  Dirección: %p", *pa, pa);

    //free resources
    free (pa);
    return EXIT_SUCCESS;
}

 void f2 (int **pa) {
  
    *pa = (int*) malloc (sizeof(int));
    return;
} 



* **Versión 3b (incorrecta)**



In [ ]:
%%writefile pp.c

#include <stdlib.h>
#include <stdio.h>


int* f2 ();

int main (int argc, char *argv[]) {
    int *pa = NULL;

    /* Why is not ok? Look the warning:
    *  warning: function returns address of local variable */
    pa = f2 ();
    if (!pa) return EXIT_FAILURE;

    *pa = 152;
    printf ("Valor: %d  Dirección: %p\n", *pa, pa);

    //free resources
    free (pa);
    return EXIT_SUCCESS;
}

 int* f2 () {
     int a;
     
     return &a;
} 

**Version 3c (Incorrecta)** [Python tutor](http://pythontutor.com/visualize.html#code=%0A%23include%20%3Cstdlib.h%3E%0A%23include%20%3Cstdio.h%3E%0A%0Avoid%20f3%20%28int%20**%29%3B%0A%0Aint%20main%20%28int%20argc,%20char%20*argv%5B%5D%29%20%7B%0A%20%20%20%20int%20*pa%20%3D%20NULL%3B%0A%0A%20%20%20%20/*%20Is%20ok%3F%20%20*/%0A%20%20%20%20f3%20%28%26pa%29%3B%0A%20%20%20%20if%20%28!pa%29%20return%20EXIT_FAILURE%3B%0A%0A%20%20%20%20*pa%20%3D%20152%3B%0A%20%20%20%20printf%20%28%22Valor%3A%20%25d%20%20Direcci%C3%B3n%3A%20%25p%22,%20*pa,%20pa%29%3B%0A%0A%20%20%20%20//free%20resources%0A%20%20%20%20free%20%28pa%29%3B%0A%20%20%20%20return%20EXIT_SUCCESS%3B%0A%7D%0A%0Avoid%20f3%20%28int%20**p%29%20%7B%0A%20%20%20%20int%20a%3B%0A%0A%20%20%20%20*p%20%3D%20%26a%3B%0A%20%20%20%20return%3B%0A%7D%0A&cumulative=false&curInstr=10&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=c&rawInputLstJSON=%5B%5D&textReferences=false)




In [ ]:
%%writefile pp.c

#include <stdlib.h>
#include <stdio.h>

void f3 (int **);

int main (int argc, char *argv[]) {
    int *pa = NULL;

    /* Is ok?  */
    f3 (&pa);
    if (!pa) return EXIT_FAILURE;

    *pa = 152;
    printf ("Valor: %d  Dirección: %p", *pa, pa);

    //free resources
    free (pa);
    return EXIT_SUCCESS;
}

void f3 (int **p) {
    int a;

    *p = &a;
    return;
}



### Array de dos dimensiones (matriz) con memoria dinámica  

En los próximos dos códigos vamos a declarar una matriz dinámicamente. Una vez reservada la memoria en ambos accederemos con la notación de array de dos dimensiones `x[i][j]`


#### Memoria dinámica no secuencial

No dejes de visualizar como se está gestionando la memoria con [Python Tutor](http://pythontutor.com/visualize.html#code=%23include%20%3Cstdlib.h%3E%0A%23include%20%3Cstdio.h%3E%0A%0Aint%20main%28%29%20%7B%0A%20%20%20%20int%20i,%20j,%20num1,%20num2%3B%0A%20%20%20%20int%20**x%3B%0A%20%20%20%20%0A%20%20%20%20num1%20%3D%205%3B%0A%20%20%20%20num2%20%3D%203%3B%0A%20%20%20%20%0A%20%20%20%20//%20Alloc%20memory%0A%20%20%20%20x%20%3D%20%28int%20**%29%20malloc%20%28num1%20*%20sizeof%28int%20*%29%29%3B%0A%20%20%20%20for%20%28i%20%3D%200%3B%20i%20%3C%20num1%3B%20i%2B%2B%29%20%7B%0A%20%20%20%20%20%20%20%20x%5Bi%5D%20%3D%20%28int%20*%29%20malloc%20%28num2%20*%20sizeof%28int%29%29%3B%0A%20%20%20%20%7D%0A%20%20%20%20%0A%20%20%20%20//%20Assign%20values%0A%20%20%20%20for%20%28i%20%3D%200%3B%20i%20%3C%20num1%20%3B%20i%2B%2B%29%20%7B%0A%20%20%20%20%20%20%20%20for%28j%20%3D%200%3B%20j%20%3C%20num2%3B%20j%2B%2B%29%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20x%5Bi%5D%5Bj%5D%20%3D%20i*100%20%2B%20j%3B%0A%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%7D%0A%20%20%20%20%0A%20%20%20%20//%20Free%20memory%0A%20%20%20%20for%20%28i%20%3D%200%3B%20i%20%3C%20num1%3B%20i%2B%2B%29%20%0A%20%20%20%20%20%20%20%20free%28%20x%5Bi%5D%29%3B%0A%20%20%20%20free%28x%29%3B%20%20%20%20%20%20%20%20%20%20%0A%7D%0A&cumulative=false&curInstr=68&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=c_gcc9.3.0&rawInputLstJSON=%5B%5D&textReferences=false)

Notad como aunque hayamos reservado memoria para una *matriz* de dimensión `num1 x num2`, y podamos acceder a ella con la notación `x[i][j]`, la memoria no es contigua como si ocurría cuando declarábamos un array de dos dimensiones con memoria estática.

In [ ]:
%%writefile pp.c

#include <stdlib.h>
#include <stdio.h>
#include <stdbool.h>

int main() {
    int i, j, num1, num2;
    int **x;
    bool bol = true;
    
    num1 = 5;
    num2 = 3;
    
    // Alloc memory
    x = (int **) malloc (num1 * sizeof(int *));
    if (!x) 
      return EXIT_FAILURE;

    for (i = 0; i < num1 && bol == true; i++) {
        x[i] = (int *) malloc (num2 * sizeof(int));
        if (x[i] == NULL)
          return bol = false;
    }
    
    // Assign values
    for (i = 0; i < num1 && bol == true; i++) {
        for(j = 0; j < num2; j++) {
            x[i][j] = i*100 + j;
        }
    }
    
    // Free memory
    for (i = 0; i < num1; i++) 
        free( x[i]);
    free(x);   

    if (bol == false)
      return EXIT_FAILURE;

    return EXIT_SUCCESS;       
}


#### Memoria dinámica secuencial
Es importante que compares como se gestiona la memoria en este ejercicio con el anterior
[Python Tutor](http://pythontutor.com/visualize.html#code=%0A%0A%23include%20%3Cstdlib.h%3E%0A%23include%20%3Cstdio.h%3E%0A%0A%0Aint%20main%28%29%20%7B%0A%20%20%20%20int%20i,%20j,%20num1,%20num2%3B%0A%20%20%20%20int%20**x%3B%0A%20%20%20%20%0A%20%20%20%20num1%20%3D%205%3B%0A%20%20%20%20num2%20%3D%203%3B%0A%20%20%20%20%0A%20%20%20%20//%20Alloc%20memory%20for%20the%20pointers%0A%20%20%20%20x%20%3D%20%28int%20**%29%20malloc%20%28num1%20*%20sizeof%28int%20*%29%29%3B%0A%20%20%20%20if%20%28!x%29%0A%20%20%20%20%20%20return%20EXIT_FAILURE%3B%0A%0A%20%20%20%20//%20memoria%20contigua%0A%20%20%20%20x%5B0%5D%20%3D%20%28int*%29%20malloc%20%28num1%20*%20num2%20*%20sizeof%28int%29%29%3B%0A%20%20%20%20if%20%28!x%5B0%5D%29%20%7B%0A%20%20%20%20%20%20%20%20free%20%28x%29%3B%0A%20%20%20%20%20%20%20%20return%20EXIT_FAILURE%3B%0A%20%20%20%20%7D%0A%0A%20%20%20%20//%20Set%20the%20pointers%0A%20%20%20%20for%20%28i%20%3D%201%3B%20i%20%3C%20num1%3B%20i%2B%2B%29%20%7B%0A%20%20%20%20%20%20%20%20x%5Bi%5D%20%3D%20x%5B0%5D%20%2B%20num2%20*i%3B%0A%20%20%20%20%7D%0A%20%20%20%20%0A%20%20%20%20//%20Assign%20values%0A%20%20%20%20for%20%28i%20%3D%200%3B%20i%20%3C%20num1%20%3B%20i%2B%2B%29%20%7B%0A%20%20%20%20%20%20%20%20for%28j%20%3D%200%3B%20j%20%3C%20num2%3B%20j%2B%2B%29%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20x%5Bi%5D%5Bj%5D%20%3D%20i*100%20%2B%20j%3B%0A%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%7D%0A%20%20%20%20%0A%20%20%20%20//%20Free%20memory%0A%20%20%20%20free%20%28x%5B0%5D%29%3B%0A%20%20%20%20free%20%28x%29%3B%20%0A%0A%20%20%20%20return%20EXIT_SUCCESS%3B%20%20%20%20%20%20%0A%7D%0A&cumulative=false&curInstr=55&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=c_gcc9.3.0&rawInputLstJSON=%5B%5D&textReferences=false)

Notad como, a diferencia del caso anterior, la memoria reservada sí es contigua

In [ ]:
%%writefile pp.c

#include <stdlib.h>
#include <stdio.h>

int main() {
    int i, j, num1, num2;
    int **x;
    
    num1 = 5;
    num2 = 3;
    
    // Alloc memory for the pointers
    x = (int **) malloc (num1 * sizeof(int *));
    if (!x)
      return EXIT_FAILURE;

    // memoria contigua
    x[0] = (int*) malloc (num1 * num2 * sizeof(int));
    if (!x[0]) {
        free (x);
        return EXIT_FAILURE;
    }

    // Set the pointers
    for (i = 1; i < num1; i++) {
        x[i] = x[0] + num2 *i;
    }
    
    // Assign values
    for (i = 0; i < num1 ; i++) {
        for(j = 0; j < num2; j++) {
            x[i][j] = i*100 + j;
        }
    }
    
    // Free memory
    free (x[0]);
    free (x); 

    return EXIT_SUCCESS;      
}


## Strings (cadenas de caracteres)

A diferencia de otros lenguajes en C no existe el tipo de dato *cadena de caracteres*. 
En `C` una cadena de caracteres es un array de caracteres donde el último caráácter de la cadena es el símbolo `\0` .

Hay varias formas de inicializar una cadena de caracteres pero hay que tener cuidado con algunas de ellas!!! [Python tutorial](http://pythontutor.com/visualize.html#code=%0A%23include%20%3Cstdlib.h%3E%0A%23include%20%3Cstdio.h%3E%0A%23include%20%3Cstring.h%3E%0A%0A%23define%20MAXCADENA%208%0A%0Aint%20main%20%28int%20argc,%20char%20*argv%5B%5D%29%20%7B%0A%20%20%20%20char%20cad1%5BMAXCADENA%5D%3B%0A%20%20%20%20char%20cad2%5B%5D%20%3D%20%7B'p',%20'e',%20'p',%20'a',%20'%5C0'%7D%3B%0A%20%20%20%20char%20*cad3%20%3D%20NULL%20%3B%0A%20%20%20%20char%20*cad4%20%3D%20NULL%3B%0A%20%20%20%20size_t%20size%3B%0A%0A%20%20%20%20strcpy%20%28cad1,%20%22hola%20%22%29%3B%20%0A%20%20%20%20strncpy%20%28cad1,%20%22hola%20%22,%20MAXCADENA-1%29%3B%0A%0A%20%20%20%20cad3%20%3D%20%28char*%29%20malloc%20%28MAXCADENA%29%3B%0A%20%20%20%20if%20%28cad3%29%0A%20%20%20%20%20%20strncpy%20%28cad3,%20%22%20paso%22,%20MAXCADENA-1%29%3B%0A%20%20%20%20%0A%20%20%20%20cad4%20%3D%20%22%20casa%3F%22%3B%20%0A%0A%20%20%20//%20fprintf%20%28stdout,%20%22%25s%25s%25s%25s%5Cn%22,%20cad1,%20cad2,%20cad3,%20cad4%29%3B%0A%0A%20%20%20%20cad1%5B0%5D%20%3D%20'H'%3B%0A%20%20%20%20cad2%5B0%5D%20%3D%20'P'%3B%0A%20%20%20%20cad3%5B0%5D%20%3D%20'P'%3B%0A%20%20%20%20/*%0A%20%20%20%20cad4%5B0%5D%20%3D%20'P'%3B%20//%20ATENTION%20NOT%20ALLOWED%0A%20%20%20%20*/%0A%0A%20%20%20%20free%20%28cad3%29%3B%0A%7D&cumulative=false&curInstr=0&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=c&rawInputLstJSON=%5B%5D&textReferences=false)



In [ ]:
%%writefile pp.c

#include <stdlib.h>
#include <stdio.h>
#include <string.h>

#define MAXCADENA 8

int main (int argc, char *argv[]) {
    char cad1[MAXCADENA];
    char cad2[] = {'p', 'e', 'p', 'a', '\0'};
    char *cad3 = NULL ;
    char *cad4 = NULL;
    size_t size;

    strcpy (cad1, "hola "); 
    strncpy (cad1, "hola ", MAXCADENA-1);

    cad3 = (char*) malloc (MAXCADENA);
    if (cad3)
      strncpy (cad3, " paso", MAXCADENA-1);
    
    cad4 = " casa?"; 

   // fprintf (stdout, "%s%s%s%s\n", cad1, cad2, cad3, cad4);

    cad1[0] = 'H';
    cad2[0] = 'P';
    cad3[0] = 'P';
    /*
    cad4[0] = 'P'; // ATENTION NOT ALLOWED
    */

    free (cad3);
}

¿Entiendes ahora la expresión `char *argv[]`?[Python tutorial](http://pythontutor.com/visualize.html#code=%23include%20%3Cstdlib.h%3E%0A%23include%20%3Cstdio.h%3E%0A%23include%20%3Cstring.h%3E%0A%0A%23define%20MAXCADENA%208%0A%0Aint%20main%20%28int%20argc,%20char%20*argv%5B%5D%29%20%7B%0A%20%20%20char%20*my_argv%5B%5D%20%3D%7B%22...%22,%20%22hola%22,%20%22don%22,%20%22pepito%22%7D%3B%0A%20%20%20int%20my_argc%20%3D%204%3B%0A%0A%20%20%20fprintf%20%28stdout,%20%22%25s%20%25s%20%25s%22,%20my_argv%5B1%5D,%20%0A%20%20%20%20%20%20%20%20%20%20%20%20my_argv%5B2%5D,%20my_argv%5B3%5D%29%3B%0A%0A%20%20%20%20return%20EXIT_SUCCESS%3B%0A%7D%0A&cumulative=false&curInstr=3&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=c&rawInputLstJSON=%5B%5D&textReferences=false) 

In [ ]:
%%writefile pp.c

#include <stdlib.h>
#include <stdio.h>
#include <string.h>

#define MAXCADENA 8

int main (int argc, char *argv[]) {
   char *my_argv[] ={"...", "hola", "don", "pepito"};
   int my_argc = 4;

   fprintf (stdout, "%s %s %s", my_argv[1], 
            my_argv[2], my_argv[3]);

    return EXIT_SUCCESS;
}



## Programación estructurada

* ¿Es correcto el siguiente código? Sigue la ejecucióón línea a línea y utiliza Valgrind [Python tutorial](http://pythontutor.com/visualize.html#code=%0A%23include%20%3Cstdlib.h%3E%0A%23include%20%3Cstdio.h%3E%0A%23include%20%3Cstdbool.h%3E%0A%0A%23define%20SIZE%203%0A%0Aint%20main%20%28int%20argc,%20char%20*argv%5B%5D%29%20%7B%0A%20%20%20%20int%20*p%5BSIZE%5D%20%3D%20%7BNULL%7D%3B%0A%20%20%20%20int%20i%3B%0A%20%20%20%20bool%20st%20%3D%20true%3B%0A%0A%20%20%20%20for%20%28i%20%3D0%3B%20i%20%3C%20SIZE%20%26%26%20st%20%3D%3D%20true%3B%20i%2B%2B%29%20%7B%0A%20%20%20%20%20%20%20%20p%5Bi%5D%20%3D%20%28int%20*%29%20malloc%20%28sizeof%28int%29%29%3B%0A%20%20%20%20%20%20%20%20if%20%28!p%5Bi%5D%29%20%0A%20%20%20%20%20%20%20%20%20%20st%20%3D%20false%3B%0A%20%20%20%20%7D%0A%20%20%20%20/*%20Error%20Control.%20If%20there%20was%20a%20problem%20with%20the%20memory%20allocation%20%0A%20%20%20%20%20*%20the%20allocated%20resources%20must%20be%20freed%0A%20%20%20%20%20**/%20%0A%20%20%20%20if%20%28i%20%3C%20SIZE%29%20%7B%0A%20%20%20%20%20%20%20%20for%20%28%20%3B%20i%20%3E%3D%200%3B%20i--%29%0A%20%20%20%20%20%20%20%20%20%20free%20%28p%5Bi%5D%29%3B%0A%20%20%20%20%20%20%20%20return%20EXIT_FAILURE%3B%0A%20%20%20%20%7D%0A%0A%20%20%20%20/*%20Assign%20values%20%20*/%0A%20%20%20%20for%20%28i%20%3D%200%20%3B%20i%20%3C%20SIZE%3B%20i%2B%2B%29%0A%20%20%20%20%20%20*%28p%5Bi%5D%29%20%3D%20100*i%3B%20%0A%0A%20%20%20%20/*%20Show%20values%20*/%0A%20%20%20%20for%20%28i%20%3D%200%20%3B%20i%20%3C%20SIZE%3B%20i%2B%2B%29%0A%20%20%20%20%20%20fprintf%20%28stdout,%20%22%25d%20%20%25d%5Cn%22,%20**%28p%2Bi%29,%20**%28p%2Bi%29%20%3D%3D%20*p%5Bi%5D%29%3B%0A%0A%20%20%20%20/*%20Free%20sources%20*/%0A%20%20%20%20for%20%28%20i%20%3D%200%20%3B%20i%20%3C%20SIZE%3B%20i%2B%2B%29%0A%20%20%20%20%20%20%20%20%20%20free%20%28p%5Bi%5D%29%3B%0A%0A%20%20%20%20//%20free%20%28p%29%3B%0A%0A%20%20%20%20return%20EXIT_SUCCESS%3B%0A%7D%0A%0A&cumulative=false&curInstr=35&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=c&rawInputLstJSON=%5B%5D&textReferences=false)

In [ ]:
%%writefile pp.c

#include <stdlib.h>
#include <stdio.h>
#include <stdbool.h>

#define SIZE 3

int main (int argc, char *argv[]) {
    int *p[SIZE] = {NULL};
    int i;
    bool st = true;

    for (i =0; i < SIZE && st == true; i++) {
        p[i] = (int *) malloc (sizeof(int));
        if (!p[i]) 
          st = false;
    }
    /* Error Control. If there was a problem with the memory allocation 
     * the allocated resources must be freed
     **/ 
    if (i < SIZE) {
        for ( ; i >= 0; i--)
          free (p[i]);
        return EXIT_FAILURE;
    }

    /* Assign values  */
    for (i = 0 ; i < SIZE; i++)
      *(p[i]) = 100*i; 

    /* Show values */
    for (i = 0 ; i < SIZE; i++)
      fprintf (stdout, "%d  %d\n", **(p+i), **(p+i) == *p[i]);

    /* Free sources */
    for ( i = 0 ; i < SIZE; i++)
          free (p[i]);

    // free (p);

    return EXIT_SUCCESS;
}



Como has podido verificar el programa anterior era *sintácticamente* correcto (la compilación y ejecución fue correcta), gestionaba correctamente la memoria (se realizó el oportuno control de errores para prevenir una posible violación de segmento si la reserva de memoria dinámica fallase) y finalmente se liberaron los recursos tal y como se mostraba en Valgrind. Pero aun así **¿Podemos considerarlo un programa correcto? No**. El programa no cumple uno de los principios básicos de la *programación  estructurada*. Debemos procurar que cada una de las acciones incluídas en el main (reserva de memoria, asignación de valores, impresión de los valores y liberación de los recursos) se realicen en diferentes funciones y en el main tan solo invoquemos a las funciones correspondientes. [Pyton tutorial](http://pythontutor.com/visualize.html#code=%0A%23include%20%3Cstdlib.h%3E%0A%23include%20%3Cstdio.h%3E%0A%23include%20%3Cstdbool.h%3E%0A%0A%23define%20SIZE%203%0A%0Atypedef%20enum%20%7BERROR%20%3D%200,%20OK%20%3D%201%7D%20Status%3B%0A%0A%0A//%20Declare%20functions%0AStatus%20array_init%20%28int%20**%29%3B%0Aint%20array_free%20%28int,%20int%20**%29%3B%0Avoid%20array_setValues%20%28int%20**%29%3B%0Aint%20array_print%20%28FILE%20*,%20int%20**%29%3B%20%0A%0A%0Aint%20main%20%28int%20argc,%20char%20*argv%5B%5D%29%20%7B%0A%20%20%20%20int%20*p%5BSIZE%5D%20%3D%20%7BNULL%7D%3B%0A%20%20%20%20int%20i%3B%0A%20%20%20%20Status%20st%20%3D%20OK%3B%0A%20%20%20%20FILE%20*pf%20%3D%20NULL%3B%0A%0A%20%20%20%20//%20Allocate%20memory%20for%20the%20array%0A%20%20%20%20st%20%3D%20array_init%20%28p%29%3B%0A%20%20%20%20if%20%28st%20%3D%3D%20ERROR%29%0A%20%20%20%20%20%20return%20array_free%20%28EXIT_FAILURE,%20p%29%3B%0A%0A%20%20%20%20//%20Set%20the%20array%20values%3B%0A%20%20%20%20array_setValues%20%28p%29%3B%0A%0A%20%20%20%20//%20Print%20the%20array%20%0A%20%20%20%20array_print%20%28stdout,%20p%29%3B%0A%0A%20%20%20%20//%20Free%20resources%0A%20%20%20%20return%20array_free%20%28EXIT_SUCCESS,%20p%29%3B%0A%7D%0A%0AStatus%20array_init%20%28int%20**p%29%20%7B%0A%20%20%20%20int%20i%3B%0A%20%20%20%20Status%20st%20%3D%20OK%3B%0A%20%20%20%20%0A%20%20%20%20for%20%28i%20%3D%200%3B%20i%20%3C%20SIZE%20%26%26%20st%20%3D%3D%20OK%3B%20i%2B%2B%29%20%7B%0A%20%20%20%20%20%20%20%20p%5Bi%5D%20%3D%20%28int%20*%29%20malloc%20%28sizeof%28int%29%29%3B%0A%20%20%20%20%20%20%20%20if%20%28!p%5Bi%5D%29%20%0A%20%20%20%20%20%20%20%20%20%20st%20%3D%20ERROR%3B%0A%20%20%20%20%7D%0A%20%20%20%20return%20st%3B%0A%7D%0A%0Aint%20array_free%20%28int%20message,%20int%20**p%29%20%7B%0A%20%20%20%20int%20i%3B%0A%20%20%20%20%0A%20%20%20%20for%20%28i%20%3D%200%3B%20i%20%3C%20SIZE%3B%20i%2B%2B%29%20%7B%0A%20%20%20%20%20%20%20%20%20%20free%20%28p%5Bi%5D%29%3B%0A%20%20%20%20%7D%0A%20%20%20%20return%20message%3B%0A%7D%0A%0Avoid%20array_setValues%20%28int%20**p%29%20%7B%0A%20%20%20%20int%20i%3B%0A%20%20%20%20%0A%20%20%20%20for%20%28i%20%3D%200%20%3B%20i%20%3C%20SIZE%3B%20i%2B%2B%29%0A%20%20%20%20%20%20*%28p%5Bi%5D%29%20%3D%20100%20*%20i%3B%20%0A%7D%20%20%20%20%20%20%0A%0Aint%20array_print%20%28FILE%20*pf,%20int%20**p%29%20%7B%0A%20%20%20%20int%20i%3B%0A%20%20%20%20int%20c%20%3D%200%3B%0A%0A%20%20%20%20for%20%28i%20%3D%200%20%3B%20i%20%3C%20SIZE%3B%20i%2B%2B%29%20%7B%0A%20%20%20%20%20%20c%20%20%3D%20c%20%2B%20fprintf%20%28pf,%20%22%25d%20%20%22,%20*p%5Bi%5D%29%3B%0A%20%20%20%20%7D%0A%20%20%20%20return%20i%3B%0A%7D%0A%20%20&cumulative=false&curInstr=42&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=c&rawInputLstJSON=%5B%5D&textReferences=false)

In [ ]:
%%writefile pp.c

#include <stdlib.h>
#include <stdio.h>
#include <stdbool.h>

#define SIZE 3

typedef enum {ERROR = 0, OK = 1} Status;


// Declare functions
Status array_init (int **);
int array_free (int, int **);
void array_setValues (int **);
int array_print (FILE *, int **); 


int main (int argc, char *argv[]) {
    int *p[SIZE] = {NULL};
    int i;
    Status st = OK;
    FILE *pf = NULL;

    // Allocate memory for the array
    st = array_init (p);
    if (st == ERROR)
      return array_free (EXIT_FAILURE, p);

    // Set the array values;
    array_setValues (p);

    // Print the array 
    array_print (stdout, p);

    // Free resources
    return array_free (EXIT_SUCCESS, p);
}

Status array_init (int **p) {
    int i;
    Status st = OK;
    
    for (i = 0; i < SIZE && st == OK; i++) {
        p[i] = (int *) malloc (sizeof(int));
        if (!p[i]) 
          st = ERROR;
    }
    return st;
}

int array_free (int message, int **p) {
    int i;
    
    for (i = 0; i < SIZE; i++) {
          free (p[i]);
    }
    return message;
}

void array_setValues (int **p) {
    int i;
    
    for (i = 0 ; i < SIZE; i++)
      *(p[i]) = 100 * i; 
}      

int array_print (FILE *pf, int **p) {
    int i;
    int c = 0;

    for (i = 0 ; i < SIZE; i++) {
      c  = c + fprintf (pf, "%d  ", *p[i]);
    }
    return i;
}
  